# A little extra!

## New addition to Week 1

### The Unreasonable Effectiveness of the Agent Loop

# What is an Agent?

## Three competing definitions

1. AI systems that can do work for you independently - Sam Altman

2. A system in which an LLM controls the workflow - Anthropic

3. An LLM agent runs tools in a loop to achieve a goal

## The third one is the new, emerging definition

But what does it mean?

Let's make it real.

In [47]:
# Start with some imports - rich is a library for making formatted text output in the terminal

from rich.console import Console
from dotenv import load_dotenv
from openai import OpenAI
import json

from openai import AzureOpenAI
import os

load_dotenv(override=True)

True

In [48]:
def show(text):
    try:
        Console().print(text)
    except Exception:
        print(text)

In [49]:
openai = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key = os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version = os.getenv("AZURE_OPENAI_API_VERSION")
)

In [50]:
# Some lists!

todos = []
completed = []

In [51]:
def get_todo_report() -> str:
    result = ""
    for index, todo in enumerate(todos):
        if completed[index]:
            result += f"Todo #{index + 1}: [green][strike]{todo}[/strike][/green]\n"
        else:
            result += f"Todo #{index + 1}: {todo}\n"
    show(result)
    return result

In [52]:
get_todo_report()

''

In [53]:
def create_todos(descriptions: list[str]) -> str:
    todos.extend(descriptions)
    completed.extend([False] * len(descriptions))
    return get_todo_report()

In [54]:
def mark_complete(index: int, completion_notes: str) -> str:
    if 1 <= index <= len(todos):
        completed[index - 1] = True
    else:
        return "No todo at this index."
    Console().print(completion_notes)
    return get_todo_report()

In [55]:
todos, completed = [], []

create_todos(["Buy groceries", "Finish extra lab", "Eat banana"])

Todo #1: Buy groceries
Todo #2: Finish extra lab
Todo #3: Eat banana

'Todo #1: Buy groceries\nTodo #2: Finish extra lab\nTodo #3: Eat banana\n'

In [56]:
mark_complete(1, "bought")

bought

Todo #1: Buy groceries
Todo #2: Finish extra lab
Todo #3: Eat banana

'Todo #1: [green][strike]Buy groceries[/strike][/green]\nTodo #2: Finish extra lab\nTodo #3: Eat banana\n'

In [57]:
create_todos_json = {
    "name": "create_todos",
    "description": "Add new todos from a list of descriptions and return the full list",
    "parameters": {
        "type": "object",
        "properties": {
            "descriptions": {
                'type': 'array',
                'items': {'type': 'string'},
                'title': 'Descriptions'
                }
            },
        "required": ["descriptions"],
        "additionalProperties": False
    }
}

In [58]:
mark_complete_json = {
    "name": "mark_complete",
    "description": "Mark complete the todo at the given position (starting from 1) and return the full list",
    "parameters": {
        'properties': {
            'index': {
                'description': 'The 1-based index of the todo to mark as complete',
                'title': 'Index',
                'type': 'integer'
                },
            'completion_notes': {
                'description': 'Notes about how you completed the todo in rich console markup',
                'title': 'Completion Notes',
                'type': 'string'
                }
            },
        'required': ['index', 'completion_notes'],
        'type': 'object',
        'additionalProperties': False
    }
}

In [59]:
tools = [{"type": "function", "function": create_todos_json},
        {"type": "function", "function": mark_complete_json}]

In [60]:
#operate the tools calling with a loop
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [64]:
def loop(messages):
    done = False
    while not done:
        response = openai.chat.completions.create(model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"), messages=messages, tools=tools)
        finish_reason = response.choices[0].finish_reason
        if finish_reason=="tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    show(response.choices[0].message.content)

In [65]:
system_message = """
You are given a problem to solve, by using your todo tools to plan a list of steps, then carrying out each step in turn.
Now use the todo list tools, create a plan, carry out the steps, and reply with the solution.
If any quantity isn't provided in the question, then include a step to come up with a reasonable estimate.
Provide your solution in Rich console markup without code blocks.
Do not ask the user questions or clarification; respond only with the answer after using your tools.
"""
user_message = """"
A train leaves Boston at 2:00 pm traveling 60 mph.
Another train leaves New York at 3:00 pm traveling 80 mph toward Boston.
When do they meet?
"""
messages = [{"role": "system", "content": system_message}, {"role": "user", "content": user_message}]

In [66]:
todos, completed = [], []
loop(messages)

Todo #1: Calculate the distance between Boston and New York to use in this problem.
Todo #2: Establish how far each train travels in a given time frame.
Todo #3: Determine when the two trains will meet by calculating their relative positions and ensuring their sum 
equals the distance between Boston and New York.

I estimated the distance between Boston and New York to be approximately 215 miles, which is a commonly known route
distance.

Todo #1: Calculate the distance between Boston and New York to use in this problem.
Todo #2: Establish how far each train travels in a given time frame.
Todo #3: Determine when the two trains will meet by calculating their relative positions and ensuring their sum 
equals the distance between Boston and New York.

The first train starts at 2:00 pm, and the second at 3:00 pm. From 2:00 pm to 3:00 pm, the first train travels 60 
miles. After 3:00 pm, both trains are moving towards each other with combined speeds of 140 mph (60 mph + 80 mph).

Todo #1: Calculate the distance between Boston and New York to use in this problem.
Todo #2: Establish how far each train travels in a given time frame.
Todo #3: Determine when the two trains will meet by calculating their relative positions and ensuring their sum 
equals the distance between Boston and New York.

After 3:00 pm, the remaining distance for the trains to cover is 215 - 60 = 155 miles. As they approach each other 
at a combined speed of 140 mph, the time it takes for them to meet is 155 / 140 ≈ 1.11 hours, or approximately 1 
hour and 7 minutes. Therefore, they meet at roughly 4:07 pm.

Todo #1: Calculate the distance between Boston and New York to use in this problem.
Todo #2: Establish how far each train travels in a given time frame.
Todo #3: Determine when the two trains will meet by calculating their relative positions and ensuring their sum 
equals the distance between Boston and New York.

The two trains will meet at approximately **4:07 pm**.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">Now try to build an Agent Loop from scratch yourself!<br/>
            Create a new .ipynb and make one from first principles, referring back to this as needed.<br/>
            It's one of the few times that I recommend typing from scratch - it's a very satisfying result.
            </span>
        </td>
    </tr>
</table>